In [1]:
import numpy as np
import pandas as pd
from numba import njit, objmode, prange,deferred_type
from numba.typed import Dict
from numba.core import types
import time
import string
from numba.typed import List as NumbaList
import matplotlib.pyplot as plt
from typing import List
from numpy import savetxt


from BPnumba.DFFA import createDFFA,DFFA
from BPnumba.EDFFA import createEDFFA,EDFFA
from BPnumba.DABC import DABC, createDABC
from BPnumba.EDABC import createEDABC ,EDABC
from BPnumba.AGH import createAG,NAG  
from BPnumba.BoxN import create_ItemBin,ItemBin,InstaceBoxes
from BPnumba.BPPH import create_Bin
from BPnumba.Individual import Ind, createR_intidivual,create_intidivual
from BPnumba.Poblation import CreateHeuristicPob,CreateRotHPob
from BPmodule.BPGen import createPA,P2A2,P3A2,P4A2,P5A2,BR1,BR2,BR3,BR4,BR5,BR6,BR7


from collections import OrderedDict
from numba.experimental import jitclass

In [2]:
bin:list[int] = NumbaList(P2A2[0])
DataSet:list[list[list[int]]] = P2A2[1][0]


UBOXES = InstaceBoxes(DataSet)
hpob = CreateRotHPob(pop =20, ITEMS = UBOXES, rotType=6)

In [3]:
for ind in hpob:
    print(ind.genome)
    print(ind.genome_r)
    print("")

[4, 2, 1, 12, 3, 10, 5, 6, 7, 13, 11, 18, 19, 22, 15, 20, 21, 14, 8, 17, 9, 24, 16, 23, 25, ...]
[5, 3, 3, 1, 0, 1, 1, 2, 1, 2, 1, 0, 2, 5, 1, 3, 1, 4, 3, 2, 0, 4, 5, 2, 5, ...]

[2, 1, 12, 10, 13, 11, 4, 5, 6, 7, 18, 19, 20, 21, 3, 8, 25, 22, 15, 14, 9, 24, 23, 17, 16, ...]
[3, 0, 4, 5, 0, 2, 1, 0, 2, 4, 1, 1, 4, 0, 4, 4, 4, 5, 4, 1, 5, 2, 5, 3, 3, ...]

[17, 9, 14, 15, 8, 3, 21, 19, 18, 16, 6, 5, 4, 20, 7, 22, 24, 10, 12, 2, 1, 11, 13, 25, 23, ...]
[0, 1, 0, 4, 3, 0, 5, 1, 5, 1, 4, 5, 0, 1, 0, 1, 4, 2, 5, 3, 1, 0, 2, 4, 4, ...]

[2, 4, 3, 1, 13, 12, 11, 10, 22, 5, 23, 6, 17, 15, 16, 7, 14, 20, 18, 24, 19, 21, 8, 9, 25, ...]
[0, 2, 0, 2, 2, 3, 1, 1, 5, 4, 1, 5, 1, 0, 5, 3, 3, 3, 4, 4, 4, 2, 4, 0, 0, ...]

[25, 24, 5, 1, 17, 20, 8, 21, 23, 19, 6, 4, 2, 7, 9, 15, 3, 14, 22, 10, 16, 13, 12, 18, 11, ...]
[5, 1, 3, 2, 4, 4, 5, 3, 0, 5, 4, 0, 4, 0, 2, 3, 5, 1, 0, 0, 2, 5, 2, 5, 4, ...]

[5, 4, 1, 11, 12, 20, 19, 3, 16, 2, 18, 8, 13, 17, 21, 15, 25, 9, 22, 14, 10, 23, 6, 24, 7, ...]
[5, 2, 0

In [ ]:
def zerovec(y, x): return np.zeros(shape=[x, y], dtype=np.float64)

createText = lambda x,y : savetxt(
    fname = "Results/"+x + ".csv",
    X= y,
    delimiter=" ",
    fmt='%g'
)

numTest = 20
num = 5
agL = zerovec(num, 2*numTest)
dabcL = zerovec(num, 2*numTest)
ffaL = zerovec(num, 2*numTest)

pr_selec = 0.85
pr_cross = 0.75
maxItr=1000
pop = 50

In [ ]:

@njit(parallel=True)
def BenchAG(boxes:List[List[List[int]]],bin:List[int],maxItr: int, lst: List[List[float]], heuristic:int=0):
    for i in prange(20):
        alg:NAG = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=0,mutType=heuristic)
        alg.Train(maxItr,  InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0), NumbaList(boxes[i]), bin)
        lst[i] =alg.BestInd.fi

def TestAG(algorithms, problems, tp):
    nm:string=""
    if tp==1:
        nm="c1HAG"
        print("Empieza",nm)
    elif tp == 2:
        nm="c2HAG"
        print("Empieza",nm)
    else:
        nm="HAG"
        print("Empieza",nm)
    for problem in problems:
        for algoritm in algorithms:
            
            agL = zerovec(1, numTest)
            contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
            BenchAG(datos.copy(),contenedor, maxItr, agL,tp)
            if problem!=1 and algoritm!=1: createText("P"+str(problem)+"A"+str(algoritm)+"/AG/"+nm,agL)

In [ ]:
@njit(parallel=True)
def BenchABC(boxes:List[List[List[int]]],bin:List[int],maxItr: int, lst: List[List[float]], heuristic:int=0):
    n=len(boxes[0])
    for i in prange(20):
        if(heuristic==0):
            alg:DABC = createDABC()
            alg.Train(numItr=maxItr,ColonyWorker=InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0),datos= NumbaList(boxes[i]),contenedor= bin)
            lst[i] =alg.BestInd.fi

        else:
            alg:EDABC = createEDABC()
            alg.Train(numItr=maxItr,ColonyWorker=InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0),datos= NumbaList(boxes[i]),contenedor= bin)
            lst[i] =alg.BestInd.fi
def TestEDABC(algorithms, problems,tp:int):
    nm:string=""
    if tp != 0:
        nm="c2EDABC"
        print("Inicia c2EDABC")
    else:
        nm="DABC"
        print("Inicia DABC")
    for problem in problems:
        for algoritm in algorithms:
            agL = zerovec(1, numTest)
            contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
            start_time = time.time()
            BenchABC(datos.copy(),contenedor, maxItr, agL,heuristic =tp)
            seconds = np.abs( start_time - time.time())
            hours = int(seconds/3600.0) 
            minutes = int(seconds/60) % 60
            sec = int(seconds % 60)
            print("termina P", problem, "A",algoritm," | ",hours,"h :",minutes,"m :",sec,"s")
            if algoritm!=1: createText("P"+str(problem)+"A"+str(algoritm)+"/ABC/"+nm,agL)
           

In [ ]:
@njit(parallel=True)
def BenchFFA(
    boxes:List[List[List[int]]],
    bin:List[int],
    maxItr: int, 
    lst: List[List[float]],
    tp:int=0
    ):
  
    for i in prange(20):
        alg:DFFA = createDFFA(mutType=0)
        alg.Train(Maxitr=maxItr,fireflyPob=InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0), datos= NumbaList(boxes[i]),contenedor= bin)
        lst[i] =alg.BestInd.fi

@njit(parallel=True)
def BenchEFFA(
    boxes:List[List[List[int]]],
    bin:List[int],
    maxItr: int, 
    lst: List[List[float]],
    tp:int=0,
    updateIndex:int=5,
    ):
   for i in prange(20): 
        alg:EDFFA = createEDFFA(mutType=0)
        alg.Train(updateIndex,maxItr, InstancePob(NumbaList( CreateHeuristicPob(pop,boxes[i],True)),boxes[i],bin),boxes[i],bin)
        lst[i] =alg.BestInd.fi

In [ ]:
def TestFFA(
    algorithms:list[int], 
    problems:list[int],
    tp:int, 
    evolutive:bool,
    updateIndex:int
    ):
    nm:string = ""
    if evolutive:
        if tp==1:
            nm="c1EDFFA"
            print("Inicia",nm)
        elif  tp==2:
            nm="c2EDFFA"
            print("Inicia", nm)
        else:
            nm="EDFFA"
            print("Inicia",nm)
    else:
        if tp==1:
            nm="c1DFFA"
            print("Inicia",nm)
        elif tp==2:
            nm="c2DFFA"
            print("Inicia",nm)
        elif tp==3:
            nm="invDFFA"
            print("Inicia",nm)
        else:
            nm="DFFA"
            print("Inicia",nm)
   
    for problem in problems:
        for algoritm in algorithms:
            agL = zerovec(1, numTest)
            contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
            start_time = time.time()
            if not evolutive:
                BenchFFA(boxes=datos,bin=contenedor, maxItr=maxItr,lst= agL,tp=tp)
            else:
                BenchEFFA(boxes=datos,bin=contenedor, maxItr=maxItr,lst= agL,tp=tp,updateIndex=updateIndex)
            seconds = np.abs( start_time - time.time())
            hours = int(seconds/3600.0) 
            minutes = int(seconds/60) % 60
            sec = int(seconds % 60)
            print("Termina P", problem, "A",algoritm," | ",hours,"h :",minutes,"m :",sec,"s")
            if algoritm!=1: createText("P"+str(problem)+"A"+str(algoritm)+"/FFA/"+nm,agL)

In [ ]:
def Compile():
    UPDATE_INDEX:int=10
    ALGORITHM=[1]
    PROBLEM=[1]
    for _ in np.arange(4):
        TestFFA(
            algorithms=ALGORITHM,
            problems=PROBLEM,
            tp=_,
            evolutive=True,
            updateIndex=UPDATE_INDEX,
        )
        if _ == 3: continue
        TestFFA(
            algorithms=ALGORITHM,
            problems=PROBLEM, 
            tp=_,
            evolutive=False,
            updateIndex=UPDATE_INDEX,
        )
    TestEDABC(ALGORITHM,PROBLEM,0)
    TestEDABC(ALGORITHM,PROBLEM,1)

In [ ]:
Compile()

In [ ]:
TestEDABC([2],[3],0)

In [ ]:

TestEDABC([2],[3],1)


In [ ]:
UPDATE_INDEX:int=10


TestFFA(
        algorithms=[4],
        problems=[2],
        tp=0,
        evolutive=True,
        updateIndex=UPDATE_INDEX
    )

TestFFA(
        algorithms=[4],
        problems=[2],
        tp=1,
        evolutive=True,
        updateIndex=UPDATE_INDEX
    )
TestFFA(
        algorithms=[4],
        problems=[2],
        tp=2,
        evolutive=True,
        updateIndex=UPDATE_INDEX
    )

In [ ]:

TestFFA(
        algorithms=[2],
        problems=[3],
        tp=2,
        evolutive=True,
        updateIndex=UPDATE_INDEX
)

TestEDABC([2],[4],0)
TestEDABC([2],[4],1)

TestFFA(
    algorithms=[2],
    problems=[4],
    tp=0,
    evolutive=True,
    updateIndex=UPDATE_INDEX
)
TestFFA(
    algorithms=[2],
    problems=[4],
    tp=1,
    evolutive=True,
    updateIndex=UPDATE_INDEX
)
TestFFA(
    algorithms=[2],
    problems=[4],
    tp=2,
    evolutive=True,
    updateIndex=UPDATE_INDEX
)
TestFFA(
        algorithms=[2],
        problems=[4],
        tp=0,
        evolutive=False,
        updateIndex=UPDATE_INDEX
)